In [108]:
import os 

os.chdir('../')
os.getcwd()

'c:\\Users\\arunk'

In [109]:
from fraudDetection.logger import logging
from fraudDetection.exception import FraudDetectionException
from fraudDetection.config.configuration import ConfigurationManager
from fraudDetection.components import DataIngestion, DataValidation, DataTransformation,ModelTrainer
from fraudDetection.entity import DataIngestionArtifact, DataValidationArtifact, DataTransformationArtifact, ModelTrainerArtifact

from fraudDetection.constants import CONFIG_FILE_PATH
config = ConfigurationManager(CONFIG_FILE_PATH)
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion_artifacts = DataIngestion(data_ingestion_config).initiate_data_ingestion()

data_validation_config =config.get_data_validation_config()
data_validation_artifacts =DataValidation(data_ingestion_artifacts,data_validation_config).initiate_data_validation()
data_transformation_config =config.get_data_transformation_config()
data_transformation_obj= DataTransformation(data_validation_artifacts,data_ingestion_artifacts,data_transformation_config)


====>>>File already exists


In [110]:
data_transformation_obj.get_data_transfomer_object()

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('cat_pipeline',
                                                  Pipeline(steps=[('ohe_type',
                                                                   OneHotEncoder())]),
                                                  BoxList(['type'])),
                                                 ('num_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  BoxList(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']))])),
                ('feature_extactor', ExtraTreesRegressor())])

In [111]:
data_transformation_obj.initiate_data_transformation()

C:\Users\arunk\FraudDetection\src\fraudDetection\components\data_transformation.py:255: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.median(),inplace=True)
C:\Users\arunk\FraudDetection\src\fraudDetection\components\data_transformation.py:255: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.median(),inplace=True)


DataTransformationArtifact(is_transformed=True, message='Transformed', transformed_train_file_path='c:\\Users\\arunk\\FraudDetection\\artifacts\\transformed_dir\\train\\file_0.npz', transformed_test_file_path='c:\\Users\\arunk\\FraudDetection\\artifacts\\transformed_dir\\test\\file_0.npz', processed_object_file_path=WindowsPath('c:/Users/arunk/FraudDetection/artifacts/transformed_dir/preprocessed/preprocessed.pkl'))

In [112]:
ing_arti = data_transformation_obj.data_ingestion_artifact
val_arti = data_transformation_obj.data_validation_artifact

In [113]:
from fraudDetection.utils import save_object, read_yaml, load_data, save_numpy_array_data
from fraudDetection.constants import *
schema_path = val_arti.schema_file_path
schema = read_yaml(schema_path)
cat_col = schema[DATA_SCHEMA_CATEGORICAL_COLUMN_KEY]
col = schema[DATA_SCHEMA_COLUMNS_KEY]
num_col = schema[DATA_SCHEMA_NUMERICAL_COLUMN_KEY]
print(col)

{'step': 'int64', 'type': 'object', 'amount': 'float64', 'oldbalanceOrg': 'float64', 'newbalanceOrig': 'float64', 'oldbalanceDest': 'float64', 'newbalanceDest': 'float64', 'isFraud': 'int64'}


In [114]:
train_df = load_data(ing_arti.train_file_path,col)
test_df = load_data(ing_arti.test_file_path,col)
target_col_name = schema[DATA_SCHEMA_TARGET_COLUMN_KEY][0]

In [516]:
input_train_df = train_df.drop(target_col_name,axis=1)
target_col = train_df[target_col_name]
print(input_train_df.shape, target_col.shape)

(39832, 7) (39832,)


In [116]:
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.base import BaseEstimator, TransformerMixin

In [521]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.feature_selection import SelectPercentile, chi2, SelectFromModel
from sklearn.utils.validation import check_is_fitted, check_X_y

In [118]:
# define the preprocessing steps for categorical and numerical columns
from sklearn.pipeline import FeatureUnion
cat_preprocessor = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    # ("selector", SelectPercentile(chi2, percentile=50)),
])

num_preprocessor = Pipeline(steps=[
    ('scaler', RobustScaler())
])

# define the preprocessor pipeline with SMOTENC and RandomOverSampler
preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_col),
    ('num', num_preprocessor, num_col)
],remainder="passthrough")



In [522]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
# SimpleImputer(missing_values=np.nan, strategy='mean')


class SMOTENCWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features:list, sampling_strategy:float,target_feature_name:str):
        self.categorical_features = categorical_features
        self.sampling_strategy = sampling_strategy
        self.target_feature_name = target_feature_name
        self.smotenc = SMOTENC(sampling_strategy=sampling_strategy, categorical_features=categorical_features)
        self.target_feature_name = target_feature_name

    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.smotenc.fit_resample(X, y)
        return self

    def transform(self, X, y=None) -> pd.DataFrame:
        df_resampled: pd.DataFrame = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return df_resampled
    
class RandomUnderSamplerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, sampling_strategy,target_feature_name):
        self.sampling_strategy = sampling_strategy
        self.sampler = RandomUnderSampler(sampling_strategy=sampling_strategy)
        self.target_feature_name = target_feature_name
    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.sampler.fit_resample(X.drop(columns=[self.target_feature_name]), X[self.target_feature_name])
        
        return self
    
    def transform(self, X, y=None) -> pd.DataFrame:
        df_resampled: pd.DataFrame = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return df_resampled
        # return self.X_resampled_, self.y_resampled_
    
class FeatureExtractor(BaseEstimator,TransformerMixin):
    def fit(self,X,y):
        self.model = ExtraTreesRegressor()
        self.model.fit(X,y)   
        return self
    def transform(self,X,y):
        return self

def get_data_balancing_object(target_feature_name) -> Pipeline:
    """
    define the preprocessor pipeline with SMOTENC and RandomUnderSampler
    Args: target feature column name
    return: pipeline object     
    """
    sampling_pipeline = Pipeline(steps=[
    ('sampling', SMOTENCWrapper(categorical_features=[1], sampling_strategy=0.1,target_feature_name=target_feature_name)),
    ('oversample', RandomUnderSamplerWrapper(sampling_strategy=0.5,target_feature_name=target_feature_name))
    ])
    return sampling_pipeline

sampling_pipeline = Pipeline(steps=[
    ('sampling', SMOTENCWrapper(categorical_features=[1], sampling_strategy=0.1,target_feature_name=target_col_name)),
    ('oversample', RandomUnderSamplerWrapper(sampling_strategy=0.5,target_feature_name=target_col_name))
    ])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, num_strategy='median', cat_strategy='most_frequent', thresh=None):
        self.num_strategy = num_strategy
        self.cat_strategy = cat_strategy
        self.thresh = thresh
        self.num_imputer = SimpleImputer(strategy=self.num_strategy)
        self.cat_imputer = SimpleImputer(strategy=self.cat_strategy)

    def fit(self, X, y):
        if self.thresh is not None:
            drop_cols = X.columns[X.isnull().mean() > self.thresh]
            X.drop(columns=drop_cols, inplace=True)

        num_cols = X.select_dtypes(include=['float', 'int']).columns
        self.num_imputer.fit(X[num_cols])

        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        self.cat_imputer.fit(X[cat_cols])
        self.y = y
        self.X = X
        return self

    def transform(self, X, y=None):
        if self.thresh is not None:
            drop_cols = X.columns[X.isnull().mean() > self.thresh]
            X.drop(columns=drop_cols, inplace=True)

        num_cols = X.select_dtypes(include=['float', 'int']).columns
        X[num_cols] = self.num_imputer.transform(X[num_cols])

        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        X[cat_cols] = self.cat_imputer.transform(X[cat_cols])

        return X, self.y

imputer = Pipeline(steps=[("CustomImputer", CustomImputer(num_strategy='median', cat_strategy='most_frequent', thresh=.3))])
# make_pipeline([(CustomImputer(num_strategy='median', cat_strategy='most_frequent', thresh=.3), set_output(transform="pandas"))])
column_name = train_df.columns[:-1]

pipe_obj = Pipeline([
    ("CustomImputer",imputer),
    # ("sampling_pipeline", sampling_pipeline),
    # ("transformer", preprocessor)
    ])

# print(train_df.columns)
pipe_obj.fit_transform(input_train_df,target_col)

In [523]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_thresh, upper_thresh):
        super().__init__()
        self.lower_thresh = lower_thresh
        self.upper_thresh = upper_thresh
 
    def fit(self, X, y):
        null_prop_ = X.isnull().sum()/len(X)
        drop_cols= null_prop_[null_prop_ > self.upper_thresh].index 
        X.drop(columns=drop_cols,inplace= True)

        drop_rows = X.loc[X.isnull().mean(axis=1).between(self.lower_thresh,self.upper_thresh)].index
        X.drop(index=drop_rows, inplace=True)
         
        return self

    def transform(self, X, y=None):
        # fill Age
        X.fillna(X.median(),inplace=True)
        return X

imputer = Pipeline([("CustomImputer",CustomImputer(lower_thresh=0.1,upper_thresh=0.3))])

# class PipelineWithY(Pipeline):
#     def transform(self, X, y=None):
#         Xt = X
#         for name, transform in self.steps[:-1]:
#             # all transformations except the last one
#             if hasattr(transform, 'transform'):
#                 Xt = transform.transform(Xt)
#         # last transformation, handle differently to get X and y
#         name, last_transform = self.steps[-1]
#         if hasattr(last_transform, 'transform'):
#             Xt = last_transform.transform(Xt)
#         # return as tuple of (X, y)
#         return Xt, y

pipe_obj = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
    # ('target_extractor', FunctionTransformer(lambda df: (df.drop(columns=target_col_name), df[target_col_name]))),
    # ("transformer", preprocessor)
])

pipe_obj.fit(input_train_df,target_col)
df_resampled =pipe_obj.transform(input_train_df)

c:\Users\arunk\FraudDetection\env\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
c:\Users\arunk\FraudDetection\env\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [524]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel

FeatureExtractor = Pipeline(steps=[
  ('feature_selection', SelectFromModel(ExtraTreesClassifier(n_estimators=50,max_features="log2"),threshold=0.05))
])

pipe_obj.fit(input_train_df,target_col)
df_resampled =pipe_obj.transform(input_train_df)

XTrain = df_resampled.iloc[:,:-1]
yTrain = df_resampled.iloc[:,-1]

cat_preprocessor = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ("selector", SelectPercentile(chi2, percentile=50)),
])

num_preprocessor = Pipeline(steps=[
    ('scaler', RobustScaler())
])

# define the preprocessor pipeline with SMOTENC and RandomOverSampler
preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_col),
    ('num', num_preprocessor, num_col)
],remainder="passthrough")

pipe_obj1 = Pipeline(steps=[
    ("transformer", preprocessor),
    ("FeatureExtractor",FeatureExtractor)
])
arr =pipe_obj1.fit_transform(XTrain,yTrain)
feature_out_name = pipe_obj1.get_feature_names_out()

c:\Users\arunk\FraudDetection\env\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
c:\Users\arunk\FraudDetection\env\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [525]:
# import numpy as np
# np.c_[arr,np.array(yTrain)]

XTrain = df_resampled.iloc[:,:-1]
yTrain = df_resampled.iloc[:,-1]
print(f"XTrain.shape: {XTrain.shape}, YTrain.shape: {yTrain.shape}" )
yTrain.values.reshape(-1,1)

XTrain.shape: (11940, 7), YTrain.shape: (11940,)


array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int64)

In [526]:
# define a function to convert the output of pipe_obj into XTrain and yTrain
def split_data(df):
    X_train = df.iloc[:,:-1]
    y_train = df.iloc[:,-1]
    print(X_train.shape,y_train.shape)
    return y_train, X_train
preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_col),
    ('num', num_preprocessor, num_col)
],remainder="passthrough")

pipe_obj1 = Pipeline(steps=[
    ("transformer", preprocessor),
    ("FeatureExtractor",FeatureExtractor)
])

# define the combined pipeline
combined_pipeline = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
    ("split_data", FunctionTransformer(split_data)),
    ("preprocessor", preprocessor),
    ("FeatureExtractor", FeatureExtractor)
])

# fit and transform the data using the combined pipeline
arr = combined_pipeline.fit_transform(input_train_df, target_col,)
feature_out_name = combined_pipeline.named_steps["FeatureExtractor"].get_feature_names_out()


c:\Users\arunk\FraudDetection\env\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


(11940, 7) (11940,)


ValueError: could not broadcast input array from shape (11940,7) into shape (11940,)

In [475]:
import numpy as np
X = np.random.random_sample((3,2))*10
print(X)
print(X[:,0])
print(X[:,0].shape)
print(X[:,0].reshape(-1,1))

[[5.232882   9.74079297]
 [4.69067116 5.22841156]
 [3.41057392 9.02033036]]
[5.232882   4.69067116 3.41057392]
(3,)
[[5.232882  ]
 [4.69067116]
 [3.41057392]]


In [335]:
df.columns.groupby(df.dtypes)
df.columns.get_loc("type")

1

In [183]:

cat_col
num_col
target_col_name
column_name
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
import numpy as np
cat_selector = make_column_selector(dtype_include=object)
num_selector =make_column_selector(dtype_include=np.number)

cat_tree_processor = OneHotEncoder()
num_tree_processor = SimpleImputer(strategy='median')

tree_processor = make_column_transformer(
    (cat_preprocessor,cat_selector),
    (num_preprocessor,num_selector))

cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
num_linear_processor = make_pipeline(
    RobustScaler(),SimpleImputer(strategy='median')
)
linear_preprocessor = make_column_transformer(
    (num_linear_processor,num_selector), (cat_linear_processor,cat_selector)
    )



In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", RobustScaler())]
)


categorical_transformer = Pipeline(
    steps=[
        # ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        # ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_col),
        ("cat", categorical_transformer, cat_col),
    ]
)
clf = Pipeline(
    steps=[("preprocessor", preprocessor)]
    )

clf.fit_transform(input_train_df,target_col)
clf.get_feature_names_out()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_thresh, upper_thresh):
        super().__init__()
        self.lower_thresh = lower_thresh
        self.upper_thresh = upper_thresh
 
    def fit(self, X, y):
        null_prop_ = X.isnull().sum()/len(X)
        drop_cols= null_prop_[null_prop_ > self.upper_thresh].index 
        X.drop(columns=drop_cols,inplace= True)

        drop_rows = X.loc[X.isnull().mean(axis=1).between(self.lower_thresh,self.upper_thresh)].index
        X.drop(index=drop_rows, inplace=True)
        self.y =y
        
        return self

    def transform(self, X, y=None):
        # fill Age
        X.fillna(X.median(),inplace=True)
        
        return X,self.y

# pipe_obj = Pipeline([
#     ("CustomImputer",imputer),
#     ("sampling_pipeline", sampling_pipeline),
#     # ('target_extractor', FunctionTransformer(lambda df: ( df[target_col_name], df.drop(columns=target_col_name)))),
#     ("ta",preprocessor)
# ])

class PipelineWithY(Pipeline):
    def transform(self, X, y=None):
        Xt = X
        for name, transform in self.steps[:-1]:
            # all transformations except the last one
            if hasattr(transform, 'transform'):
                Xt = transform.transform(Xt)
        # last transformation, handle differently to get X and y
        name, last_transform = self.steps[-1]
        if hasattr(last_transform, 'transform'):
            Xt = last_transform.transform(Xt)
        # return as tuple of (X, y)
        return Xt, y
    
pipe_obj = PipelineWithY([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
    ('target_extractor', FunctionTransformer(lambda df: ( df[target_col_name], df.drop(columns=target_col_name)))),
    ("ta",preprocessor)
])
pipe_obj.fit(input_train_df,target_col)
pipe_obj.transform(input_train_df)

In [ ]:


enc = OneHotEncoder()
enc_data = pd.DataFrame(enc.fit_transform(
    input_train_df[['type']]).toarray())
  
# Merge with main
New_df = input_train_df.join(enc_data)
New_df.drop(columns=['type'],inplace=True)
names = enc.get_feature_names_out()
New_df.select_dtypes(include='integer').columns

# OH_X_train= pd.concat([New_df,enc_data], axis=1)
# OH_X_train

Index(['step'], dtype='object')

In [ ]:
from sklearn.utils.validation import check_is_fitted, check_X_y
class GroupImputer(BaseEstimator, TransformerMixin):
    '''
    Class used for imputing missing values in a pd.DataFrame using either mean or median of a group.
    
    Parameters
    ----------    
    group_cols : list
        List of columns used for calculating the aggregated value 
    target : str
        The name of the column to impute
    metric : str
        The metric to be used for remplacement, can be one of ['mean', 'median']
    Returns
    -------
    X : array-like
        The array with imputed values in the target column
    '''
    def __init__(self, group_cols, target, metric='mean'):
        
        assert metric in ['mean', 'median'], 'Unrecognized value for metric, should be mean/median'
        assert type(group_cols) == list, 'group_cols should be a list of columns'
        assert type(target) == str, 'target should be a string'
        
        self.group_cols = group_cols
        self.target = target
        self.metric = metric
    
    def fit(self, X, y=None):
        
        assert pd.isnull(X[self.group_cols]).any(axis=None) == False, 'There are missing values in group_cols'
        
        impute_map = X.groupby(self.group_cols)[self.target].agg(self.metric) \
                                                            .reset_index(drop=False)
        
        self.impute_map_ = impute_map
        
        return self 
    
    def transform(self, X, y=None):
        
        # make sure that the imputer was fitted
        check_is_fitted(self, 'impute_map_')
        
        X = X.copy()
        
        for index, row in self.impute_map_.iterrows():
            ind = (X[self.group_cols] == row[self.group_cols]).all(axis=1)
            X.loc[ind, self.target] = X.loc[ind, self.target].fillna(row[self.target])
        
        return X.values
    
GroupImputer(group_cols, target, metric='median')

In [ ]:
with config_context(transform_output="pandas"):
    # the output of transform will be a Pandas DataFrame
    X_test_scaled = scaler.transform(X_test[num_cols])
X_test_scaled.head()
